In [1]:
import pandas as pd
import numpy as np
from astropy.io import fits
import astropy
import math
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import colorcet as cc
from acstools import acszpt
from astropy.table import Table
%matplotlib qt


In [2]:
#m = -2.5 x log10(DN / EXPTIME) + ZEROPOINT

WFC3

In [3]:
#import image data
image_data_w814_fireball, header_w814_fireball = fits.getdata("C:/Users/willi/Desktop/Hubble Data/candidates/2/h_v75_F814W_ivm_drz_cl_ver2.fits", header=True)
image_data_w475_fireball, header_w475_fireball = fits.getdata("C:/Users/willi/Desktop/Hubble Data/candidates/2/h_v75_F475W_ivm_drz_cl_ver2.fits", header=True)

#get full fits file
hdu_list = fits.open("C:/Users/willi/Desktop/Hubble Data/candidates/2/h_v75_F814W_ivm_drz_cl_ver2.fits")
image_data = hdu_list[1].data
hdu_list.info()

Filename: C:/Users/willi/Desktop/Hubble Data/candidates/2/h_v75_F814W_ivm_drz_cl_ver2.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     503   ()      
  1  SCI           1 ImageHDU       179   (4225, 4300)   float32   
  2  WHT           1 ImageHDU        71   (4225, 4300)   float32   
  3  CTX           1 ImageHDU        71   (4225, 4300)   int32   


In [4]:
image_data_w814_fireball

array([[-0.01948908, -0.06138653,  0.00473011, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.01288939, -0.01733125,  0.01123847, ...,  0.        ,
         0.        ,  0.        ],
       [-0.02398009,  0.01929787,  0.00155553, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -0.00296626,
         0.00758985,  0.        ],
       [ 0.        ,  0.        ,  0.        , ..., -0.00940749,
         0.02341227,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.01901661,
         0.02487982,  0.        ]], dtype=float32)

In [8]:
i = []
for char in image_data_w814_fireball:
    i.append(min(char))

In [10]:
min(i)

-0.3722971

In [11]:
image_data_w814_fireball = image_data_w814_fireball + 0.3722971

In [12]:
#display plot of data using a log scale
plt.imshow(image_data_w814_fireball, cmap= "Blues", interpolation = "nearest", norm=LogNorm())
plt.colorbar()

In [13]:
date = '2007-05-08'
detector = 'WFC'

In [14]:
#queary zeropoints
q814 = acszpt.Query(date=date, detector=detector, filt = "F814W")                                    
filter_zpt814 = q814.fetch()

q475 = acszpt.Query(date=date, detector=detector, filt = "F475W")                                    
filter_zpt475 = q475.fetch()



INFO:acszpt.Query:Checking inputs...
INFO:acszpt.Query:Submitting request to https://acszeropoints.stsci.edu/results_single/?date1=2007-05-08&detector=WFC&WFC_filter=F814W
INFO:acszpt.Query:Parsing the response and formatting the results...
INFO:acszpt.Query:Checking inputs...
INFO:acszpt.Query:Submitting request to https://acszeropoints.stsci.edu/results_single/?date1=2007-05-08&detector=WFC&WFC_filter=F475W
INFO:acszpt.Query:Parsing the response and formatting the results...


In [15]:
print(filter_zpt814)
print(filter_zpt475)

FILTER PHOTPLAM        PHOTFLAM         STmag  VEGAmag  ABmag 
       Angstrom erg / (Angstrom cm2 s) mag(ST)   mag   mag(AB)
------ -------- ---------------------- ------- ------- -------
 F814W   8045.3              7.137e-20  26.766  25.503   25.93
FILTER PHOTPLAM        PHOTFLAM         STmag  VEGAmag  ABmag 
       Angstrom erg / (Angstrom cm2 s) mag(ST)   mag   mag(AB)
------ -------- ---------------------- ------- ------- -------
 F475W   4747.5              1.852e-19  25.731  26.133  26.041


In [16]:
#convert to AB mag
m_ab_w_814_fireball = (-2.5 * np.log10(image_data_w814_fireball)) + filter_zpt814['ABmag'][0].value

m_ab_w_475_fireball = (-2.5 * np.log10(image_data_w475_fireball)) + filter_zpt475['ABmag'][0].value

<ipython-input-16-d0dec94f65b6>:2: RuntimeWarning: divide by zero encountered in log10
  m_ab_w_814_fireball = (-2.5 * np.log10(image_data_w814_fireball)) + filter_zpt814['ABmag'][0].value
<ipython-input-16-d0dec94f65b6>:4: RuntimeWarning: divide by zero encountered in log10
  m_ab_w_475_fireball = (-2.5 * np.log10(image_data_w475_fireball)) + filter_zpt475['ABmag'][0].value
<ipython-input-16-d0dec94f65b6>:4: RuntimeWarning: invalid value encountered in log10
  m_ab_w_475_fireball = (-2.5 * np.log10(image_data_w475_fireball)) + filter_zpt475['ABmag'][0].value


In [17]:
m_ab_w_475_fireball

array([[      nan, 30.541595, 31.79977 , ...,       inf,       inf,
              inf],
       [32.98281 , 30.889114, 31.943262, ...,       inf,       inf,
              inf],
       [31.755981, 32.24063 , 32.156128, ...,       inf,       inf,
              inf],
       ...,
       [      inf,       inf,       inf, ..., 30.608961, 31.046036,
              inf],
       [      inf,       inf,       inf, ..., 30.778755, 32.771294,
              inf],
       [      inf,       inf,       inf, ..., 30.8269  ,       nan,
              inf]], dtype=float32)

In [18]:
m_ab_w_814_fireball_array = np.where(m_ab_w_814_fireball == np.inf, np.nan, m_ab_w_814_fireball)
m_ab_w_475_fireball_array = np.where(m_ab_w_475_fireball == np.inf, np.nan, m_ab_w_475_fireball)

In [19]:
print(m_ab_w_814_fireball_array)

[[27.061153 27.198412 26.989069 ... 27.002777 27.002777 27.002777]
 [26.965822 27.054533 26.970486 ... 27.002777 27.002777 27.002777]
 [27.075064 26.947906 26.99825  ... 27.002777 27.002777 27.002777]
 ...
 [27.002777 27.002777 27.002777 ... 27.011461 26.980864 27.002777]
 [27.002777 27.002777 27.002777 ... 27.030563 26.93656  27.002777]
 [27.002777 27.002777 27.002777 ... 26.948689 26.93254  27.002777]]


In [20]:
subtract_coma = m_ab_w_475_fireball_array - m_ab_w_814_fireball_array

In [21]:
plt.imshow(m_ab_w_814_fireball_array , interpolation = "nearest", cmap= "Blues")
plt.clim(21,32)#for wide
#plt.clim(25.5,30.5)#for core
#plt.clim(-1,2) #for subtract
#plt.clim(-1,1) #for subtract core
plt.colorbar()

Using photutils for photometry

In [15]:
from photutils.aperture import CircularAperture

In [16]:
positions = [(30., 30.), (40., 40.)]
aperture = CircularAperture(positions, r=3.)

In [17]:
aperture

<CircularAperture([[30., 30.],
                   [40., 40.]], r=3.0)>

WFPC2

In [37]:
image_data_w814_spikes, header_w814_spikes = fits.getdata("C:/Users/willi/Desktop/Hubble Data/candidates/1/hst_06773_03_wfpc2_f814w_pc_sci.fits", header=True)
image_data_w450_spikes, header_w450_spikes = fits.getdata("C:/Users/willi/Desktop/Hubble Data/candidates/1/hst_06773_03_wfpc2_f450w_pc_sci.fits", header=True)

#ZPAB=−2.5∗log10(PHOTFLAM)−5∗log10(PHOTPLAM)−2.408

zp_ab_450 = (-2.5 * np.log10(1.267181601123595 * (10**-18))) - (5 * np.log10(4557.367)) - 2.408
zp_ab_814 = (-2.5 * np.log10(3.522445224719101 * (10**-19))) - (5 * np.log10(7995.943)) - 2.408

m_ab_450 = -2.5 * np.log10(image_data_w450_spikes) + zp_ab_450
m_ab_814 = -2.5 * np.log10(image_data_w814_spikes) + zp_ab_814

subtract = m_ab_450 - m_ab_814

<ipython-input-37-e132a3b8b280>:9: RuntimeWarning: divide by zero encountered in log10
  m_ab_450 = -2.5 * np.log10(image_data_w450_spikes) + zp_ab_450
<ipython-input-37-e132a3b8b280>:9: RuntimeWarning: invalid value encountered in log10
  m_ab_450 = -2.5 * np.log10(image_data_w450_spikes) + zp_ab_450
<ipython-input-37-e132a3b8b280>:10: RuntimeWarning: divide by zero encountered in log10
  m_ab_814 = -2.5 * np.log10(image_data_w814_spikes) + zp_ab_814
<ipython-input-37-e132a3b8b280>:10: RuntimeWarning: invalid value encountered in log10
  m_ab_814 = -2.5 * np.log10(image_data_w814_spikes) + zp_ab_814
<ipython-input-37-e132a3b8b280>:12: RuntimeWarning: invalid value encountered in subtract
  subtract = m_ab_450 - m_ab_814


In [19]:
header_w814_spikes

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 1000                                                  
NAXIS2  =                 1000                                                  
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
CRVAL1  =    195.4994698105556 / right ascension of reference pixel (deg)       
CRVAL2  =    27.78257197472222 / declination of reference pixel (deg)           
CRPIX1  =                500.0 / x-coordinate of reference pixel                
CRPIX2  =                500.0 / y-coordinate of reference pixel                
CD1_1   = -1.388888888888889E-05 / partial of the right ascension w.r.t. x      
CD1_2   =                  0

In [38]:
plt.imshow(m_ab_450, cmap= "Blues", interpolation = "nearest")
#plt.clim(22,30) #all
plt.clim(23.5,27) #spikes

plt.colorbar()

In [40]:
plt.imshow(subtract, cmap= "Blues", interpolation = "nearest")
plt.clim(-.3,2)
plt.colorbar()

In [25]:
plt.imshow(image_data_w814_spikes, cmap= "Blues", interpolation = "nearest", norm=LogNorm())
plt.colorbar()